In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('Sample.csv')

In [30]:
import numpy as np

wa_func = lambda x: np.average(x, weights=df.loc[x.index, 'Balance'])

df.pivot_table(
    index=['Counterparty', 'Contract'],
    columns='Delivery',
    values=['Balance', 'Price'],
    aggfunc={
        'Balance': sum,
        'Price': wa_func
    },
    margins=True
).fillna('').swaplevel(0,1,axis=1).sort_index(axis=1).round(3)

Delivery              1/8/2019               All         
                       Balance    Price  Balance    Price
Counterparty Contract                                    
A            A1         200.00  134.000   200.00  134.000
             A2         133.44  134.000   133.44  134.000
             A3         500.00  132.147   500.00  132.147
B            B1          54.87  151.000    54.87  151.000
             B2         200.00  149.000   200.00  149.000
C            C1         500.00  150.000   500.00  150.000
             C2          20.66  135.000    20.66  135.000
             C3         100.00  135.000   100.00  135.000
             C4         100.00  147.000   100.00  147.000
D            D1        1324.05  134.566  1324.05  134.566
E            E1         279.87  153.000   279.87  153.000
             E2         200.00  151.000   200.00  151.000
F            F1         110.15  149.000   110.15  149.000
G            G           35.04  151.000    35.04  151.000
All                    3758.08  140.424  3758.08  140.424

In [10]:
df_grouped = df.groupby(['Counterparty', 'Contract', 'Delivery']).apply(lambda x: pd.Series(
                {
                    'Balance': x['Balance'].sum(),
                    'Price': np.average(x['Price'], weights=x['Balance']),
                }
                )).round(3).unstack().swaplevel(0,1, axis=1).sort_index(axis=1)

df_grouped

Delivery              1/8/2019         
                       Balance    Price
Counterparty Contract                  
A            A1         200.00  134.000
             A2         133.44  134.000
             A3         500.00  132.147
B            B1          54.87  151.000
             B2         200.00  149.000
C            C1         500.00  150.000
             C2          20.66  135.000
             C3         100.00  135.000
             C4         100.00  147.000
D            D1        1324.05  134.566
E            E1         279.87  153.000
             E2         200.00  151.000
F            F1         110.15  149.000
G            G           35.04  151.000

In [29]:
# Applying different aggregations on the data
df.groupby('Counterparty')['Balance', 'Price'].agg({'Balance' : ['sum', 'count'], 
                                                    'Price':['max', 'std']})

Balance          Price          
                  sum count      max       std
Counterparty                                  
A              833.44     3  134.000  1.069830
B              254.87     2  151.000  1.414214
C              720.66     4  150.000  7.889867
D             1324.05     1  134.566       NaN
E              479.87     2  153.000  1.414214
F              110.15     1  149.000       NaN
G               35.04     1  151.000       NaN

In [20]:
# Define a lambda function to compute the weighted mean:
wm = lambda x: np.average(x, weights=df.loc[x.index, "Balance"])

# Define a dictionary with the functions to apply for a given column:
f = {'Balance': ['sum'], 'Price': {'WAP' : wm} }

# Groupby and aggregate with your dictionary:
df.groupby(['Counterparty', 'Contract', 'Delivery']).agg(f).unstack().swaplevel(0,2,axis=1).swaplevel(1,2, axis=1)

Delivery              1/8/2019         
                       Balance    Price
                           sum      WAP
Counterparty Contract                  
A            A1         200.00  134.000
             A2         133.44  134.000
             A3         500.00  132.147
B            B1          54.87  151.000
             B2         200.00  149.000
C            C1         500.00  150.000
             C2          20.66  135.000
             C3         100.00  135.000
             C4         100.00  147.000
D            D1        1324.05  134.566
E            E1         279.87  153.000
             E2         200.00  151.000
F            F1         110.15  149.000
G            G           35.04  151.000